In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import PrepDataHelper as helper
from scipy.stats.mstats import zscore
from scipy.stats import norm
import pymc3 as pm
import scipy
import pymc3_models as models

In [ ]:
def GetPhobiaScoreList(df,phobia_string, survey_data):
    score_arr = []
    for index, row in df.iterrows():
        phobia_score = survey_data.loc[survey_data.Q11 == row.participant_num][phobia_string].values
        if(len(phobia_score)>0):
            score_arr.append(phobia_score[0])
        else:
            score_arr.append(np.NAN)
    return score_arr

physio_file = 'full_logfile_scr_3.csv'
df = pd.read_csv(physio_file)
survey_data = pd.read_csv('phobia_scores')

df = df.drop(df.loc[df.rt_fear>df.rt_fear.mean()+3*df.rt_fear.std()].index)

#recoded for negative valence is closer to one
df['hp_change_video']=df['video_hp']-df['base_ECG']
df['scr_change_video']=df['video_scr']-df['base_scr']

df['spider_phobia'] = GetPhobiaScoreList(df,'spider_avg',survey_data)# helper.AssignPhobiaScore(phobia_spider, df.participant_num)
df['social_phobia'] = GetPhobiaScoreList(df,'social_avg',survey_data)#helper.AssignPhobiaScore(phobia_social, df.participant_num)
df['heights_phobia']= GetPhobiaScoreList(df,'heights_avg',survey_data)
df = helper.TakeAwayGroupsLessThenX(df, min_groups=12)
df.groupby('participant_num')[['hp_change_video','scr_change_video']].mean().to_csv('groups_takenaway.csv')

In [ ]:
for par,group_df in df.groupby('participant_num'):
    print(par)
    


In [ ]:
standardized_fear_scr_hp  = df[['hp_change_video','participant_num','scr_change_video','resp_fear']].groupby('participant_num').transform(lambda x: (x - x.mean()) / x.std())
standardized_df = pd.concat([standardized_fear_scr_hp, df[['participant_num','video_condition']]], axis=1)
standardized_df = standardized_df.loc[df.video_condition.isin([1,2,3])]
full_df = standardized_df

print(full_df[['hp_change_video','participant_num']].dropna()['participant_num'].nunique())
print(full_df[['scr_change_video','participant_num']].dropna()['participant_num'].nunique())

## Individual Models SCR

In [ ]:
y = 'resp_fear'
situation_subj_model_scr = models.RegressOnePredictor_SituationSubjectHierarchical(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

subj_hierarchical_model_scr = models.RegressOnePredictor_SubjectHierarchical(
    full_df[['scr_change_video','resp_fear','participant_num']],
    y,
    'scr_change_video')

flat_model_scr = models.RegressOnePredictor(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

situation_model_scr = models.RegressOnePredictor_Situation(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

participant_model_scr = models.RegressOnePredictor_Idiographic(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

independent_situation_participant_scr =models.RegressOnePredictor_SituationSubjectIndependent(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y,
    'scr_change_video')

meanmodel_scr = models.MeanModel(
    full_df[['scr_change_video','resp_fear','participant_num','video_condition']],
    y)

In [ ]:
n_samples = 10000
flat_model_trace_scr = models.SampleModel(flat_model_scr, n_samples)
situation_model_trace_scr = models.SampleModel(situation_model_scr, n_samples)
subj_hierarchical_model_trace_scr = models.SampleModel(subj_hierarchical_model_scr, n_samples)
situation_subj_model_trace_scr = models.SampleModel(situation_subj_model_scr, n_samples)
participant_model_trace_scr = models.SampleModel(participant_model_scr, n_samples)
participant_situation_model_trace_scr = models.SampleModel(independent_situation_participant_scr, n_samples)
mean_model_trace_scr =  models.SampleModel(meanmodel_scr, n_samples)

pm.plot_posterior(subj_hierarchical_model_trace_scr, var_names = ['mu_scr_change_video'])

scr_compare = pm.compare({
        'flat_model_scr': flat_model_trace_scr,
        'subj_hierarchical_model_scr':subj_hierarchical_model_trace_scr,
        'situation_subj_model_scr':situation_subj_model_trace_scr,
        'situation_model_scr': situation_model_trace_scr,
        'participant_model_scr':participant_model_trace_scr,
        'independent_situation_participant_scr':participant_situation_model_trace_scr,
        'mean_model':mean_model_trace_scr
    }, 
        ic='LOO',scale='negative_log')
#calculate akaike weight using equation from Wagenmakers and Farell (2004)
scr_compare.weight = helper.CalcAkaikeWeights(scr_compare.d_loo)

In [ ]:
# calc model comparison mean model vs flat model
scr_mean_null =pm.compare({
                    'flat_model_scr': flat_model_trace_scr,
                    'mean_model':mean_model_trace_scr
                }, 
                    ic='LOO',scale='negative_log')

scr_mean_null.weight = helper.CalcAkaikeWeights(scr_mean_null.d_loo)

In [ ]:
#save tables
scr_mean_null.to_csv('scr_ic_table_null_vs_mean.csv')
scr_compare.to_csv('scr_ic_table.csv')

In [ ]:
# do the same analysis but with HP
y = 'resp_fear'
situation_subj_model_hp = models.RegressOnePredictor_SituationSubjectHierarchical(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

subject_hierarchical_model_hp = models.RegressOnePredictor_SubjectHierarchical(
    full_df[['hp_change_video','resp_fear','participant_num']],
    y,
    'hp_change_video')

flat_model_hp = models.RegressOnePredictor(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

situation_model_hp = models.RegressOnePredictor_Situation(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

participant_model_hp = models.RegressOnePredictor_Idiographic(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

independent_situation_participant_hp =models.RegressOnePredictor_SituationSubjectIndependent(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y,
    'hp_change_video')

meanmodel_hp = models.MeanModel(
    full_df[['hp_change_video','resp_fear','participant_num','video_condition']],
    y)

In [ ]:

participant_model_trace_hp = models.SampleModel(participant_model_hp, n_samples)
participant_situation_model_trace_hp = models.SampleModel(independent_situation_participant_hp, n_samples)
mean_model_trace_hp =  models.SampleModel(meanmodel_hp, n_samples)
flat_model_trace_hp = models.SampleModel(flat_model_hp, n_samples)
situation_model_trace_hp = models.SampleModel(situation_model_hp, n_samples)
subject_hierarchical_model_trace_hp = models.SampleModel(subject_hierarchical_model_hp, n_samples)
situation_subj_model_trace_hp = models.SampleModel(situation_subj_model_hp, n_samples)

pm.plot_posterior(subject_hierarchical_model_hp, var_names = ['mu_scr_change_video'])

hp_compared = pm.compare({
    'General': flat_model_trace_hp,
    'Subject Hierarchical':subject_hierarchical_model_trace_hp,
    'Situation Hierarchical':situation_subj_model_trace_hp,
    'Situation':situation_model_trace_hp,
    'participant_model_hp':participant_model_trace_hp,
    'independent_situation_participant_hp':participant_situation_model_trace_hp,
    'mean_model':mean_model_trace_hp
}, ic='LOO',scale='Negative_log')

hp_compared.weight = helper.CalcAkaikeWeights(hp_compared.d_loo)

In [ ]:
mean_vs_null_hp = pm.compare({
        'General': flat_model_trace_hp,
        'mean_model':mean_model_trace_hp
    }, ic='LOO',scale='Negative_log',method='pseudo-BMA')
mean_vs_null_hp.weight = helper.CalcAkaikeWeights(mean_vs_null_hp.d_loo)

In [ ]:
mean_vs_null_hp.to_csv('hp_ic_table_null_vs_mean.csv')
hp_compared.to_csv('hp_ic_table.csv')

# Correlation analyses

In [ ]:
def CorrelateBetaWithPhobia(trace,
                            beta_str,
                            phobia_str,
                            situation_num,
                            full_df,
                            df,
                            predictor_var):
    mean_betas = np.mean(trace[beta_str][:,:,situation_num],axis=0)
    #then we need to get a list of all participants from those betas
    full_df_dropped = full_df[[predictor_var,
                            'resp_fear',
                            'participant_num',
                            'video_condition']].dropna()
    participant_list = full_df_dropped.participant_num.unique()
    phobes = df.loc[df['participant_num'].isin(participant_list)][['participant_num',phobia_str]]
    #only one phobia score per participant (repeated) - just used mean as a lazy way to reduce it single score
    par_phobes = phobes.groupby('participant_num').mean().reset_index()[phobia_str]
    phobe_mean_df = pd.DataFrame(data = {
            'phobe_scores':par_phobes,
            'mean_beta':mean_betas
    })
    phobe_mean_df_noNA = phobe_mean_df.dropna()
    return stats.pearsonr(phobe_mean_df_noNA['phobe_scores'],phobe_mean_df_noNA['mean_beta'])



In [ ]:
#individual differences
#first we need to get all the mean height beta
corrs_and_ps = {}
corrs_and_ps['hp_social_phobia']= CorrelateBetaWithPhobia(situation_subj_model_trace_hp,'beta_hp_change_video',
                        'social_phobia',1,full_df,df,'hp_change_video')
corrs_and_ps['hp_spider_phobia']=CorrelateBetaWithPhobia(situation_subj_model_trace_hp,'beta_hp_change_video',
                        'spider_phobia',2,full_df,df,'hp_change_video')
corrs_and_ps['hp_heights_phobia'] = CorrelateBetaWithPhobia(situation_subj_model_trace_hp,'beta_hp_change_video',
                        'heights_phobia',0,full_df,df,'hp_change_video')

corrs_and_ps['scr_social_phobia'] = CorrelateBetaWithPhobia(situation_subj_model_trace_scr,'beta_scr_change_video',
                        'social_phobia',1,full_df,df,'scr_change_video')
corrs_and_ps['scr_spider_phobia']=CorrelateBetaWithPhobia(situation_subj_model_trace_scr,'beta_scr_change_video',
                        'spider_phobia',2,full_df,df,'scr_change_video')
corrs_and_ps['scr_heights_phobia'] =CorrelateBetaWithPhobia(situation_subj_model_trace_scr,'beta_scr_change_video',
                        'heights_phobia',0,full_df,df,'scr_change_video')

phobia,r,p=[],[],[]
for key in corrs_and_ps.keys():
    phobia.append(key)
    r.append(corrs_and_ps[key][0])
    p.append(corrs_and_ps[key][1])
    
corr_phobe_df = pd.DataFrame({'phobia':phobia,'r':r,'p':p})
corr_phobe_df.to_csv('phobia_correlation_csv')

# Visualizations 

# Plotting

In [ ]:
import numpy as np

from arviz.rcparams import rcParams
import arviz as az
from arviz.plots.plot_utils import get_plotting_function

#borrowed this method from pymc3 compare - copied in to make some small changes (font size etc)
def plot_compare(
    comp_df,
    insample_dev=True,
    plot_standard_error=True,
    plot_ic_diff=True,
    order_by_rank=True,
    figsize=None,
    textsize=None,
    plot_kwargs=None,
    ax=None,
    backend=None,
    backend_kwargs=None,
    show=None,
):
    """
    Summary plot for model comparison.

    This plot is in the style of the one used in the book Statistical Rethinking (Chapter 6)
    by Richard McElreath.

    Notes
    -----
    Defaults to comparing Leave-one-out (psis-loo) if present in comp_df column,
    otherwise compares Widely Applicable Information Criterion (WAIC)


    Parameters
    ----------
    comp_df : pd.DataFrame
        Result of the `az.compare()` method
    insample_dev : bool, optional
        Plot in-sample deviance, that is the value of the information criteria without the
        penalization given by the effective number of parameters (pIC). Defaults to True
    plot_standard_error : bool, optional
        Plot the standard error of the information criteria estimate. Defaults to True
    plot_ic_diff : bool, optional
        Plot standard error of the difference in information criteria between each model
         and the top-ranked model. Defaults to True
    order_by_rank : bool
        If True (default) ensure the best model is used as reference.
    figsize : tuple, optional
        If None, size is (6, num of models) inches
    textsize: float
        Text size scaling factor for labels, titles and lines. If None it will be autoscaled based
        on figsize.
    plot_kwargs : dict, optional
        Optional arguments for plot elements. Currently accepts 'color_ic',
        'marker_ic', 'color_insample_dev', 'marker_insample_dev', 'color_dse',
        'marker_dse', 'ls_min_ic' 'color_ls_min_ic',  'fontsize'
    ax: axes, optional
        Matplotlib axes or bokeh figures.
    backend: str, optional
        Select plotting backend {"matplotlib","bokeh"}. Default "matplotlib".
    backend_kwargs: bool, optional
        These are kwargs specific to the backend being used. For additional documentation
        check the plotting method of the backend.
    show : bool, optional
        Call backend show function.

    Returns
    -------
    axes : matplotlib axes or bokeh figures


    Examples
    --------
    Show default compare plot

    .. plot::
        :context: close-figs

        >>> import arviz as az
        >>> model_compare = az.compare({'Centered 8 schools': az.load_arviz_data('centered_eight'),
        >>>                  'Non-centered 8 schools': az.load_arviz_data('non_centered_eight')})
        >>> az.plot_compare(model_compare)

    Plot standard error and information criteria difference only

    .. plot::
        :context: close-figs

        >>> az.plot_compare(model_compare, insample_dev=False)

    """
    if plot_kwargs is None:
        plot_kwargs = {}

    yticks_pos, step = np.linspace(0, -1, (comp_df.shape[0] * 2) - 1, retstep=True)
    yticks_pos[1::2] = yticks_pos[1::2] + step / 2

    yticks_labels = [""] * len(yticks_pos)

    _information_criterion = ["loo", "waic"]
    column_index = [c.lower() for c in comp_df.columns]
    for information_criterion in _information_criterion:
        if information_criterion in column_index:
            break
    else:
        raise ValueError(
            "comp_df must contain one of the following"
            " information criterion: {}".format(_information_criterion)
        )

    if order_by_rank:
        comp_df.sort_values(by="rank", inplace=True)

    compareplot_kwargs = dict(
        ax=ax,
        comp_df=comp_df,
        figsize=figsize,
        plot_ic_diff=plot_ic_diff,
        plot_standard_error=plot_standard_error,
        insample_dev=insample_dev,
        yticks_pos=yticks_pos,
        yticks_labels=yticks_labels,
        plot_kwargs=plot_kwargs,
        information_criterion=information_criterion,
        textsize=textsize,
        step=step,
        backend_kwargs=backend_kwargs,
        show=show,
    )

    if backend is None:
        backend = rcParams["plot.backend"]
    backend = backend.lower()

    # TODO: Add backend kwargs
   # plot = get_plotting_function("plot_compare", "compareplot", backend)
    ax = mpl_plot_compare(**compareplot_kwargs)

    return ax
import matplotlib.pyplot as plt

from arviz.plots.plot_utils import _scale_fig_size
from arviz.plots.backends.matplotlib import backend_kwarg_defaults, backend_show, create_axes_grid


def mpl_plot_compare(
    ax,
    comp_df,
    figsize,
    plot_ic_diff,
    plot_standard_error,
    insample_dev,
    yticks_pos,
    yticks_labels,
    plot_kwargs,
    information_criterion,
    textsize,
    step,
    backend_kwargs,
    show,
):
    """Matplotlib compare plot."""
    if backend_kwargs is None:
        backend_kwargs = {}

    backend_kwargs = {
        **backend_kwarg_defaults(),
        **backend_kwargs,
    }

    if figsize is None:
        figsize = (6, len(comp_df))

    figsize, ax_labelsize, _, xt_labelsize, linewidth, _ = _scale_fig_size(figsize, textsize, 1, 1)
    
    linewidth = 5
    
    backend_kwargs.setdefault("figsize", figsize)
    backend_kwargs["squeeze"] = True

    if ax is None:
        _, ax = create_axes_grid(1, backend_kwargs=backend_kwargs)

    if plot_ic_diff:
        yticks_labels[0] = comp_df.index[0]
        yticks_labels[2::2] = comp_df.index[1:]
        ax.set_yticks(yticks_pos)
        ax.errorbar(
            x=comp_df[information_criterion].iloc[1:],
            y=yticks_pos[1::2],
            xerr=comp_df.dse[1:],
            color=plot_kwargs.get("color_dse", "grey"),
            fmt=plot_kwargs.get("marker_dse", "^"),
            mew=linewidth,
            elinewidth=linewidth,
        )

    else:
        yticks_labels = comp_df.index
        ax.set_yticks(yticks_pos[::2])

    if plot_standard_error:
        ax.errorbar(
            x=comp_df[information_criterion],
            y=yticks_pos[::2],
            xerr=comp_df.se,
            color=plot_kwargs.get("color_ic", "k"),
            fmt=plot_kwargs.get("marker_ic", "o"),
            mfc="None",
            mew=linewidth,
            lw=linewidth,
        )
    else:
        ax.plot(
            comp_df[information_criterion],
            yticks_pos[::2],
            color=plot_kwargs.get("color_ic", "k"),
            marker=plot_kwargs.get("marker_ic", "o"),
            mfc="None",
            ms=15,
            mew=linewidth,
            lw=0,
        )

    if insample_dev:
        ax.plot(
            comp_df[information_criterion] - (2 * comp_df["p_" + information_criterion]),
            yticks_pos[::2],
            color=plot_kwargs.get("color_insample_dev", "k"),
            ms=15,
            marker=plot_kwargs.get("marker_insample_dev", "o"),
            mew=linewidth,
            lw=0,
        )

    ax.axvline(
        comp_df[information_criterion].iloc[0],
        ls=plot_kwargs.get("ls_min_ic", "--"),
        color=plot_kwargs.get("color_ls_min_ic", "grey"),
        lw=linewidth,
    )
    
    print(yticks_pos)

    scale_col = information_criterion + "_scale"
    if scale_col in comp_df:
        scale = comp_df[scale_col].iloc[0].capitalize()
    else:
        scale = "Deviance"
    ax.set_xlabel(scale, fontsize=ax_labelsize)
    ax.set_yticklabels(yticks_labels)
    ax.set_ylim(-1 + step, 0 - step)
    ax.tick_params(labelsize=xt_labelsize)

    if backend_show(show):
        plt.show()

    return ax

In [ ]:
#These are based on scr_compare_ic csv - but are just converting the model names into the names used in the paper
scr_compare.index = ['Situation Hierarchical','Situation',
                     'Situation Participant','Participant Hierarchical','General','Null',
                    'Participant']
scr_compare.iloc[5,0]=100
plot = plot_compare(scr_compare,
             plot_standard_error=False,
             figsize=(24,8),textsize=30,
             plot_ic_diff=True,
             backend='matplotlib',
)
plot.axhline(-.91,lw=5,c='grey')
plot.xaxis.set_label_text('Negative Log Likelihood')

In [ ]:
hp_compared.iloc[4,0]=100 ## Set mean model to be last model for visualization purposes
hp_compared.index=['Situation Hierarchical','Situation',
                     'Participant Hierarchical','General','Situation Participant','Participant','Null']

plot = plot_compare(hp_compared,
             plot_standard_error=False,
             figsize=(24,8),textsize=30,
             plot_ic_diff=True,
             backend='matplotlib',
           # backend_kwargs={'linewidth':10,'marker_size':50}
)
plot.axhline(-.91,lw=5,c='grey')
plot.xaxis.set_label_text('Negative Log Likelihood')


In [ ]:
from scipy.stats import norm
x = np.arange(-1.25,1.25,.001)
plt.figure(figsize=(10,8))
colors = ['#42AC65', '#F39C12', '#9C6CC7']
situation_label = ['Height','Social','Spider']

## situation hierarchical
plt.subplot(221)
scr_means = np.mean(
    situation_subj_model_trace_scr['beta_scr_change_video'],
    axis=0
)
scr_std = np.std(
    situation_subj_model_trace_scr['beta_scr_change_video'],
    axis=0
)


for situation in range(scr_means.shape[1]):
    for participant in range(scr_means.shape[0]):
        par_mean =scr_means[participant,situation]
        par_std = scr_std[participant,situation]
        par_normal = norm.pdf(
            x,
            par_mean,
            par_std,
        )
        plt.plot(x,par_normal,alpha=.1,c=colors[situation])
plt.title('Situation Hierarchical') 

## SITUATION MODEL
plt.subplot(222)
scr_means = np.mean(
    situation_model_trace_scr['beta_scr_change_video'],
    axis=0
)
scr_std = np.std(
    situation_model_trace_scr['beta_scr_change_video'],
    axis=0
)


for situation in range(scr_means.shape[0]):
    par_mean =scr_means[situation]
    par_std = scr_std[situation]
    par_normal = norm.pdf(
        x,
        par_mean,
        par_std,
    )
    plt.plot(x,par_normal,lw=1,c=colors[situation],label=situation_label[situation])
plt.legend()
plt.title('Situation')
## situation participant
plt.subplot(223)
scr_means = np.mean(
    participant_situation_model_trace_scr['beta_scr_change_video'],
    axis=0
)
scr_std = np.std(
    participant_situation_model_trace_scr['beta_scr_change_video'],
    axis=0
)

colors = ['#42AC65', '#F39C12', '#9C6CC7']
for situation in range(scr_means.shape[1]):
    for participant in range(scr_means.shape[0]):
        par_mean =scr_means[participant,situation]
        par_std = scr_std[participant,situation]
        par_normal = norm.pdf(
            x,
            par_mean,
            par_std,
        )
        plt.plot(x,par_normal,alpha=.2,c=colors[situation])
plt.title('Situation Participant')  

###### PARTICIPNAT HIERARICHAL
plt.subplot(224)
scr_means = np.mean(
    subj_hierarchical_model_trace_scr['beta_scr_change_video'],
    axis=0
)
scr_std = np.std(
    subj_hierarchical_model_trace_scr['beta_scr_change_video'],
    axis=0
)

for participant in range(scr_means.shape[0]):
    par_mean =scr_means[participant]
    par_std = scr_std[participant]
    par_normal = norm.pdf(
        x,
        par_mean,
        par_std,
    )
    plt.plot(x,par_normal,alpha=.1,c='black')
plt.title('Participant Hierarchical')


In [ ]:
from scipy.stats import norm
x = np.arange(-1.25,1.25,.001)
plt.figure(figsize=(10,8))
colors = ['#42AC65', '#F39C12', '#9C6CC7']

## situation hierarchical
plt.subplot(221)
hp_means = np.mean(
    situation_subj_model_trace_hp['beta_hp_change_video'],
    axis=0
)
hp_std = np.std(
    situation_subj_model_trace_hp['beta_hp_change_video'],
    axis=0
)


for situation in range(hp_means.shape[1]):
    for participant in range(hp_means.shape[0]):
        par_mean =hp_means[participant,situation]
        par_std = hp_std[participant,situation]
        par_normal = norm.pdf(
            x,
            par_mean,
            par_std,
        )
        plt.plot(x,par_normal,alpha=.1,c=colors[situation])
plt.title('Situation Hierarchical')    
plt.subplot(222)
hp_means = np.mean(
    situation_model_trace_hp['beta_hp_change_video'],
    axis=0
)
hp_std = np.std(
    situation_model_trace_hp['beta_hp_change_video'],
    axis=0
)


for situation in range(hp_means.shape[0]):
    par_mean =hp_means[situation]
    par_std = hp_std[situation]
    par_normal = norm.pdf(
        x,
        par_mean,
        par_std,
    )
    plt.plot(x,par_normal,lw=1,c=colors[situation],label=situation_label[situation])
plt.legend()
plt.title('Situation')

# PARTICIPANT SITUATION
plt.subplot(223)
hp_means = np.mean(
    participant_situation_model_trace_hp['beta_hp_change_video'],
    axis=0
)
hp_std = np.std(
    participant_situation_model_trace_hp['beta_hp_change_video'],
    axis=0
)

colors = ['#42AC65', '#F39C12', '#9C6CC7']
for situation in range(hp_means.shape[1]):
    for participant in range(hp_means.shape[0]):
        par_mean =hp_means[participant,situation]
        par_std = hp_std[participant,situation]
        par_normal = norm.pdf(
            x,
            par_mean,
            par_std,
        )
        plt.plot(x,par_normal,alpha=.2,c=colors[situation])
plt.title('Situation Participant')
###### PARTICIPANT HIERARCHICAL
plt.subplot(224)
hp_means = np.mean(
    subject_hierarchical_model_trace_hp['beta_hp_change_video'],
    axis=0
)
hp_std = np.std(
    subject_hierarchical_model_trace_hp['beta_hp_change_video'],
    axis=0
)

for participant in range(hp_means.shape[0]):
    par_mean =hp_means[participant]
    par_std = hp_std[participant]
    par_normal = norm.pdf(
        x,
        par_mean,
        par_std,
    )
    plt.plot(x,par_normal,alpha=.1,c='black')
plt.title('Participant Hierarchical')


In [ ]:
def PlotRegressionWithMeanHDIIntervals(condition_idx,
                                       colors,
                                       situation_names,
                                       predictor,
                                       trace):
    beta_mean = np.mean(trace[f'mu_{predictor}_change_video'][:,condition_idx])
    b_mean = np.mean(trace['mu_b'][:,condition_idx])
    lin = np.linspace(-3,3,100)
    plt.plot(lin,lin*beta_mean+b_mean,color=colors[condition_idx],lw=2)
    low_hdi,high_hdi = pm.stats.hdi(trace[f'mu_{predictor}_change_video'][:,condition_idx])

    conf = np.zeros(shape=(2,100))
    conf[0,:]=lin*low_hdi+b_mean
    conf[1,:]=lin*high_hdi+b_mean
    low_conf = np.min(conf,axis=0)
    high_conf = np.max(conf,axis=0)

    plt.fill_between(lin,
                     low_conf,high_conf,alpha=.25,
                     color=colors[condition_idx],
                     label=situation_names[condition_idx])
situation_names = ['Height','Social','Spider']
colors = ['#42AC65', '#F39C12', '#9C6CC7']

for condition_idx in range(3):
    plt.subplot(1,2,1)
    PlotRegressionWithMeanHDIIntervals(condition_idx,colors,situation_names,'scr',
                                       situation_subj_model_trace_scr)
    plt.ylim(-3,3)
    plt.xlabel('Standardized SCR')
    plt.ylabel('Standardized Fear')
    
for condition_idx in range(3):
    plt.subplot(1,2,2)
    PlotRegressionWithMeanHDIIntervals(condition_idx,colors,situation_names,'hp',
                                       situation_subj_model_trace_hp)
    plt.legend()
    plt.xlabel('Standardized HP')
    plt.ylim(-3,3)